In [85]:
# GỘP FILE
import pandas as pd

files = ["shopee_reviews_1.csv", "shopee_reviews_2.csv", "shopee_reviews_3.csv", "shopee_reviews_4.csv", "shopee_reviews_5.csv"]

df_list = []

for f in files:
    df = pd.read_csv(f)
    df_list.append(df)

merged_df = pd.concat(df_list, ignore_index=True)

merged_df.to_csv("shopee_reviews.csv", index=False, encoding="utf-8-sig")

print("Đã gộp xong! File lưu thành: shopee_reviews.csv")


Đã gộp xong! File lưu thành: shopee_reviews.csv


In [ ]:
df = pd.read_csv("shopee_reviews.csv")
print("----Info----")
print(df.info())

print("----Null----")
print(df.isnull().sum())

print("----Duplicate----")
print(df.duplicated(subset=["user", "comment"]).sum())

# all_dupes = df[df.duplicated(subset=['user','comment'],keep=False)]
# print(all_dupes)

# null_rows = df[df.isnull().any(axis=1)]
# print(null_rows)

----Info----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     1946 non-null   object
 1   stars    1968 non-null   int64 
 2   comment  1926 non-null   object
 3   info     1968 non-null   object
dtypes: int64(1), object(3)
memory usage: 61.6+ KB
None
----Null----
user       22
stars       0
comment    42
info        0
dtype: int64
----Duplicate----
20


In [ ]:
import pandas as pd
from underthesea import word_tokenize
import re

df = pd.read_csv("shopee_reviews.csv")

# Tách cột info thành 3 cột date, time, category
df[['datetime_part', 'category_part']] = df['info'].str.split('|', expand=True)
df['datetime_part'] = df['datetime_part'].str.strip()
df['category_part'] = df['category_part'].str.strip()
df[['date', 'time']] = df['datetime_part'].str.split(' ', expand=True)
df['category'] = df['category_part'].str.replace("Phân loại hàng:", "", regex=False).str.strip()
df = df.drop(columns=['datetime_part', 'category_part', 'info'])

# Kiểm tra và xử lý missing values
df = df.dropna(subset=["comment"])     # bỏ dòng không có comment

# Loại bỏ trùng lặp
df = df.drop_duplicates()
df = df.drop_duplicates(subset=["user", "comment"])

# Chuẩn hóa comment
df['comment_clean'] = (
    df['comment']
        .astype(str)
        .str.lower()
        .str.replace(r'[^\w\s]', ' ', regex=True)  #loại dấu câu, ký tự đặc biệt (thay bằng space)
        .str.replace(r'\s+', ' ', regex=True)      #loại bỏ nhiều khoảng trắng liên tiếp
        .str.strip()
)

# Tokenization
text_series = df['comment_clean']

df['tokens'] = text_series.apply(lambda x: word_tokenize(x, format="text"))

# chuẩn hóa thủ công một số từ phổ biến trong review Shopee
# phần này viết càng nhiều mấy từ viết tắt thì data càng chuẩn
normalize_dict = {
    "k": "không",
    "kh": "không",
    "ko": "không",
    "k0": "không",
    "kg": "không",
    "kô": "không",
    "hok": "không",
    "khong": "không",
    "kp": "không phải",
    "mn": "mọi người",
    "mik": "mình",
    "mik": "mình",
    "đc": "được",
    "dc": "được",
    "đk": "được",
    "ok": "ổn",
    "oke": "ổn",
    "sp": "sản phẩm",
    "nv": "nhân viên",
    "shop": "cửa hàng",
    "ktra": "kiểm tra"
}

def normalize_tokens(text):
    words = text.split()
    normalized = [normalize_dict.get(w, w) for w in words]
    return " ".join(normalized)

df['tokens_normalized'] = df['tokens'].apply(normalize_tokens)

df.to_csv("shopee_reviews_cleaned.csv", index=False, encoding="utf-8-sig")
print("Tiền xử lý cơ bản hoàn tất. Lưu vào shopee_reviews_cleaned.csv")

Tiền xử lý cơ bản hoàn tất. Lưu vào shopee_reviews_cleaned.csv
